In [ ]:
#!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, StackingRegressor, VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor

dftrain = pd.read_csv('/train.csv')
dftest = pd.read_csv('/test.csv')
#samp = pd.read_csv('/sample_submission.csv')

print(len(dftrain.columns))
print(len(dftest.columns))

81
80


In [ ]:
# o número mais baixo é 1, e NA (valor ausente) é 0 ->
# verifiquei que NA no texto de descrição se refere a valores ausentes (somente para alguns valores)
# talvez considerar uma nova coluna para valores ausentes, ao invés do valor 0? nah
numeric_categories = {
    'Utilities':['ELO','NoSeWa','NoSewr','AllPub'],
    'HeatingQC':['Po','Fa','TA','Gd','Ex'],
    'LotShape':['IR3','IR2','IR1','Reg'],
    'LandContour':['Low','HLS','Bnk','Lvl'],
    'LandSlope':['Sev','Mod','Gtl'],
    'ExterQual':['Po','Fa','TA','Gd','Ex'],
    'ExterCond':['Po','Fa','TA','Gd','Ex'],
    'BsmtQual':['Po','Fa','TA','Gd','Ex'],#
    'BsmtCond':['Po','Fa','TA','Gd','Ex'],#
    'BsmtExposure':['No','Mn','Av','Gd'],#
    'BsmtFinType1':['Unf','LwQ','Rec','BLQ','ALQ','GLQ'],#
    'BsmtFinType2':['Unf','LwQ','Rec','BLQ','ALQ','GLQ'],#
    'KitchenQual':['Po','Fa','TA','Gd','Ex'],
    'Functional':['Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'],
    'FireplaceQu':['Po','Fa','TA','Gd','Ex'],#
    'GarageQual':['Po','Fa','TA','Gd','Ex'],#
    'GarageCond':['Po','Fa','TA','Gd','Ex'],#
    'GarageFinish':['Unf','RFn','Fin'],#
    'PoolQC':['Fa','TA','Gd','Ex'],#
    'CentralAir':['N','Y'],
    'Street':['Grvl','Pave'],
    # 'Alley':['NA','Grvl','Pave'],
    #'Fence':['NA','MnWw','GdWo','MnPrv','GdPrv'], #NA omitido?
    # 'LotConfig':['Inside','Corner','CulDSac','FR2','FR3'], #testar essa
    # 'HouseStyle':['1Story','1.5Unf','1.5Fin','2Story','2.5Unf','2.5Fin','SFoyer','SLvl'], #testar essa
    'PavedDrive':['N','P','Y'],
    #'Electrical':['FuseP','FuseF','FuseA','Mix','SBrkr'],
    #'BldgType':['1Fam','2fmCon','Duplex','TwnhsE','Twnhs'] #testar
}

In [ ]:
square_values = [
    'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','LotArea','GrLivArea',
    'MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF'
]

In [ ]:
#dftrain.groupby(['Street','PavedDrive']).size()

In [ ]:
#dftrain['MSZoning'].isin(['FV','RH','RL','RP','RM']).value_counts()

In [ ]:
# arr = np.array([1,np.NaN,2])
# np.sqrt(arr)

In [ ]:
#dftrain['FireplaceQu'].value_counts(dropna=False)
#len(dftrain[pd.isnull(dftrain['FireplaceQu'])])

In [ ]:
def clean_data(df):
    clean_df = df.copy()
    for col in square_values:
      clean_df[col] = np.sqrt(clean_df[col])

    #necessário caso eu esteja imputando o valor mais frequente
    # clean_df['MiscFeature'].fillna('missing',inplace=True)
    # clean_df['Alley'].fillna('missing',inplace=True)
    # clean_df['Fence'].fillna('missing',inplace=True)
    # clean_df['Has_Garage'] = np.where(pd.isnull(clean_df['GarageType']),0,1)

    clean_df['Zoning_Residential'] = np.where(clean_df['MSZoning'].isin(['FV','RH','RL','RP','RM']), 1, 0)
    clean_df['SaleType_Warranty'] = np.where(clean_df['SaleType'].isin(['WD','CWD','VWD']), 1, 0)
    clean_df['SaleType_Contract'] = np.where(clean_df['SaleType'].isin(['Con','ConLw','ConLI','ConLD']), 1, 0)
    clean_df['Had_Remodelling'] = np.where(clean_df['YearRemodAdd'] == clean_df['YearBuilt'], 0, 1) #a coluna original continua necessária?
    clean_df['Has_Basement'] = np.where(pd.isnull(clean_df['BsmtExposure']),0,1)
    clean_df['Has_Pool'] = np.where(pd.isnull(clean_df['PoolQC']),0,1)
    clean_df['Has_Fireplace'] = np.where(pd.isnull(clean_df['FireplaceQu']),0,1)
    # teoria (não foi 100% comprovada mas parece ser): para as categorias numericas,
    # é melhor ter uma coluna falando se aquela feature existe ou não, pois somente o valor
    # '0' da categoria numérica não indica se existe
    # teoria -> daria pra ignorar os 0s nas categorias numericas e deixar somente a coluna
    # que diz se existe ou não ter essa informação?
    elec_list = {'FuseP':1,'FuseF':2,'FuseA':3,'Mix':0,'SBrkr':0}
    clean_df['ElecQuality'] = clean_df['Electrical'].replace(elec_list).astype('Int64')
    clean_df['Electrical'] = np.where(clean_df['Electrical'].isin(['FuseP','FuseF','FuseA']),'Fuse',clean_df['Electrical'])

    for col in numeric_categories.keys():
      val_array = numeric_categories[col]
      sub_dict = {val: val_array.index(val) for val in val_array} # n é preciso dividir pois isso será feito no pipeline
      #print(sub_dict)
      clean_df[col].replace(sub_dict, inplace=True)
      # if val_array[0] == 'NA':
      #   clean_df[col].fillna(0,inplace=True)
      clean_df[col].fillna(0,inplace=True)
      # os casos para preencher com a mediana são lidadas pelo pipeline do modelo
      # aqui lidamos apenas com os valores onde ausente significa algo (falta de sótao, por exemplo),
      # e não quando realmente é ausente
      clean_df[col] = clean_df[col].astype('Int64')
      # Int64 para poder deixar valores NaN sem dar erro (senão precisariamos de errors="ignore")

    return clean_df

In [ ]:
# função não mais usada, substituida pelos imputers do sklearn para
# reproduzir isso durante o treinamento do modelo em cross_validation no Pipeline
def impute_data(df, cols):
  clean_df = df.copy()

  for col in cols:
    col_type = clean_df[col].dtype
    #print(col_type)
    if col_type == 'object':
      most_frequent = clean_df[col].value_counts().index[0]
      #clean_df[col].fillna(most_frequent, inplace=True)
      dummies = pd.get_dummies(clean_df[col], dtype=int,prefix=col,dummy_na=True)
      clean_df = pd.concat([clean_df.drop(col,axis=1),dummies],axis=1)
    else:
      val = clean_df[col].mean()
      if col_type == 'int64':
        val = clean_df[col].median()
      clean_df[col].fillna(val, inplace=True)
  # por algum motivo, se fazer todos juntos o df ficará com 2 colunas SalePrice
  return clean_df

In [ ]:
dftrainclean = clean_data(dftrain)
dftestclean = clean_data(dftest)

#já esta tendo overfitting, mesmo que não seja perfeito é bem melhor deixar em numeros
# dftrainclean['MSSubClass'] = dftrainclean['MSSubClass'].astype(str)
# dftestclean['MSSubClass'] = dftestclean['MSSubClass'].astype(str)
#print(dftrainclean.head())

# (isso só se aplicava antes quando a imputação era feita antes do teste CV)
# as categorias que não estão presentes no dataset de testes acabam por fazer com que
# existam colunas somente no dataset de treino. Precisamos apenas retirar essas colunas
# do dataset de treino (menos a de preço), já que elas não serão usadas mesmo, senão
# acontecerá um erro do shape de entrada quando o modelo for fazer as predições

In [ ]:
#dftrain['Electrical'].value_counts()

In [ ]:
#dftrainclean['Electrical'].value_counts()

In [ ]:
#dftrainclean['MSSubClass'].dtype #dtype('int64')
# pode ser um problema? considerar mudar para variável categórica, não sei se os números fazem sentido

In [ ]:
#print(dftrainclean.columns)
#print(dftestclean.columns)
# não será mais usado já que o imputing esta sendo feito no sklearn
diff = []
for col in list(dftrainclean.columns):
  if col not in list(dftestclean.columns):
    diff.append(col)
    print(col)
diff.remove('SalePrice')
dftrainclean = dftrainclean.drop(diff,axis=1)

SalePrice


In [ ]:
# log de todas as variáveis com preço (não aplicavel para possiveis valores 0)
# dftrainclean['MiscVal'] = np.log(dftrainclean['MiscVal'])
# dftestclean['MiscVal'] = np.log(dftestclean['MiscVal'])

In [ ]:
def create_pipeline(model):
  preprocessor = ColumnTransformer(
      transformers=[
          ('mean_imputer', SimpleImputer(strategy='mean'), selector(dtype_include=['float64'])), # KNNImputer, iterativeimputer, ...
          ('median_imputer', SimpleImputer(strategy='median'), selector(dtype_include=['int64'])), # iterativeimputer, ...
          ('categorical', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Replace NaN with a placeholder
            #('imputer', SimpleImputer(strategy='most_frequent')), # parece ter mais overfitting
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
          ]), selector(dtype_include=['object'])),
    ])

  pipeline = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('stack', model)
  ])
  return pipeline

In [ ]:
x = dftrainclean.drop(['SalePrice','Id'],axis=1)
y = np.log(dftrainclean['SalePrice'])

#'mle' é um algoritmo para selecionar automaticamente, existem um gap
# muito perceptível na variância das features, e o número parece estar bem correto
# todas as outras são significativamente menores

#pca = create_pipeline(PCA(n_components='mle'))
#new_features = pd.DataFrame(pca.fit_transform(x,y))
#print(x.shape)
#print(new_features.shape)
# print(pca['stack'].explained_variance_)

# x_train, x_test, y_train, y_test = train_test_split(new_features,y,random_state=9)
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=9)

In [ ]:
gbr = GradientBoostingRegressor(random_state=9, max_depth=2, n_estimators=300)
gbr1 = GradientBoostingRegressor(random_state=9, max_depth=1, n_estimators=50)
#rfr = RandomForestRegressor(random_state=9, max_depth=1, n_estimators=50)
#mlp = MLPRegressor(solver='lbfgs',max_iter=300)
ridge = Ridge(random_state=9, alpha=30)
xgboost = XGBRegressor(random_state=9,verbosity=0,max_depth=4)
# cat = CatBoostRegressor(random_state=9)
# usamos o pipeline no ensemble

#por padrão o modelo final é um ridge
# model = StackingRegressor([('gbr', gbr),('xgb', xgboost),('linear', ridge)])
model = create_pipeline(StackingRegressor([('gbr', gbr),('xgb', xgboost),('linear', ridge),('gbr1',gbr1)]))

model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('mean_imputer',
                                                  SimpleImputer(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7c276617de70>),
                                                 ('median_imputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7c276617df60>),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute...
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=4,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=9, ...)),
                                               ('linear',
                                                Ridge(alpha=30,
                                                      random_state=9)),
                                               ('gbr1',
                                                GradientBoostingRegressor(max_depth=1,
                                                                          n_estimators=50,
                                                                          random_state=9))]))])

In [ ]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
train_error = MSE(y_train, y_train_pred)**(1/2)
test_error = MSE(y_test, y_test_pred)**(1/2)
print(train_error)
print(test_error)
#train~0.07057 e test~0.11359
#train~0.07077 e test~0.11266
#train~0.08019 e test~0.11391
# na real isso nem é preciso, pois é a mesma coisa que o que está sendo feito no CV

0.08020353374930562
0.11401410882807503


In [ ]:
model = create_pipeline(StackingRegressor([('gbr', gbr),('xgb', xgboost),('linear', ridge),('gbr1',gbr1)]))

# cv_results = cross_validate(model, new_features, y, scoring='neg_root_mean_squared_error',cv=6, return_train_score=True)
cv_results = cross_validate(model, x, y, scoring='neg_root_mean_squared_error',cv=6, return_train_score=True)
# que nem cross_val_score, mas retorna também o score de treinamento para detectar overfitting

In [ ]:
RMSLE_train = np.mean(-cv_results['train_score'])
RMSLE_test = np.mean(-cv_results['test_score'])
print(-cv_results['train_score'])
print(-cv_results['test_score'])
print(RMSLE_train)
print(RMSLE_test)
# 0.06966 e 0.12238 para 0.12145
# 0.06834 e 0.12188 para 0.12080
# 0.07078 e 0.12163 para 0.12024
# 0.07087 e 0.12156 para 0.11849 (teoria -> retirando zeros (únicos) de NAs -> retirar redundancia)
# usar cvs e random_state para mais consistência

[0.06938109 0.06231003 0.0685085  0.07261294 0.07484088 0.07759011]
[0.10481757 0.1087046  0.14954083 0.12051447 0.10724624 0.1385892 ]
0.07087392695781791
0.12156881930508807


In [ ]:
gbr2 = GradientBoostingRegressor(random_state=9, max_depth=2, n_estimators=300)
gbr12 = GradientBoostingRegressor(random_state=9, max_depth=1, n_estimators=50)
ridge2 = Ridge(random_state=9, alpha=30)
xgboost2 = XGBRegressor(random_state=9,verbosity=0,max_depth=4)

model = create_pipeline(StackingRegressor([('gbr', gbr2),('xgb', xgboost2),('linear', ridge2),('gbr1',gbr12)]))

param_grid = {
    #'stack__gbr__max_depth':[1,2,3,4,5],
    #'stack__gbr__n_estimators':[100,150,200,250,300,350,400,450,500],
    'stack__xgb__eta':[0.3,0.28,0.26,0.24,0.22,0.2],
    #'stack__linear__alpha':[20,25,30,35,40,45],
}
# #grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_root_mean_squared_error',random_state=9)
# grid_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=5, n_iter=20, scoring='neg_root_mean_squared_error',random_state=9,return_train_score=True)
# grid_search.fit(x, y)

In [ ]:
#print(grid_search.cv_results_)
# print(-grid_search.best_score_)
# print(grid_search.best_params_)

In [ ]:
# params_dict = {'rank':list(grid_search.cv_results_['rank_test_score']), 'test_score':grid_search.cv_results_['mean_test_score'], 'train_score':grid_search.cv_results_['mean_train_score']}
# for key in param_grid.keys():
#   params_dict[key] = list(grid_search.cv_results_['param_'+key])
# all_parameters = pd.DataFrame(params_dict)
# all_parameters['overfitting'] = np.abs(all_parameters['test_score'] - all_parameters['train_score'])
# all_parameters.sort_values(by='overfitting',inplace=True)
# all_parameters.set_index('rank',inplace=True)
# all_parameters.head(n=10)

In [ ]:
model = create_pipeline(StackingRegressor([('gbr', gbr),('xgb', xgboost),('linear', ridge),('gbr1',gbr1)]))

# model.fit(new_features, y)
model.fit(x,y)

# posso usar todos do treinamento para treinar, já que antes eu
# não conseguiria se quisesse testar o modelo no próprio dataset fornecido
#print(x.shape)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('mean_imputer',
                                                  SimpleImputer(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7c2773f3b640>),
                                                 ('median_imputer',
                                                  SimpleImputer(strategy='median'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7c2773f3b5e0>),
                                                 ('categorical',
                                                  Pipeline(steps=[('impute...
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=4,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=9, ...)),
                                               ('linear',
                                                Ridge(alpha=30,
                                                      random_state=9)),
                                               ('gbr1',
                                                GradientBoostingRegressor(max_depth=1,
                                                                          n_estimators=50,
                                                                          random_state=9))]))])

In [ ]:
x_final = dftestclean.drop(['Id'],axis=1)
# y_pred = np.exp(model.predict(pd.DataFrame(pca.transform(x_final))))
y_pred = np.exp(model.predict(x_final))

df_submission = pd.DataFrame({'Id':dftestclean['Id'],'SalePrice':y_pred})
df_submission.set_index('Id',inplace=True)
df_submission.to_csv('/sub36.csv')
df_submission.shape

(1459, 1)